In [ ]:
import pandas as pd

In [ ]:
!pip install -q transformers==4.40.2

In [ ]:
import transformers
print(transformers.__version__)


In [ ]:
# Load sentiment classification dataset
sentiment_df = pd.read_parquet("hf://datasets/FinGPT/fingpt-sentiment-cls/data/train-00000-of-00001-921f33f83a4110cb.parquet")

# Load FiQA QA dataset
qa_df = pd.read_parquet("hf://datasets/FinGPT/fingpt-fiqa_qa/data/train-00000-of-00001-ab79bf9300210e98.parquet")

In [ ]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git
%%capture
%cd /content/IndicTrans2/huggingface_interface
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers==4.53.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece




In [ ]:
# 1️⃣ Re-clone (if not already there)
!rm -rf IndicTransToolkit
!git clone https://github.com/VarunGumma/IndicTransToolkit.git

# 2️⃣ Move into it and install properly (editable install breaks after restart)
%cd IndicTransToolkit
!pip install .

# 3️⃣ Go back to parent folder
%cd ..

# 4️⃣ Verify installation
import IndicTransToolkit
from IndicTransToolkit.processor import IndicProcessor
print("✅ IndicTransToolkit imported successfully!")


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig
from IndicTransToolkit.processor import IndicProcessor

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None  # keep as None unless you want 8-bit or 4-bit loading
BATCH_SIZE = 4       # adjust based on GPU memory

# ✅ Use the rotary model with 2048-token context
en_indic_ckpt_dir = "prajdabre/rotary-indictrans2-en-indic-dist-200M"

def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
        torch_dtype=torch.float16,
    )

    if qconfig is None:
        model = model.to(DEVICE)

    model.eval()
    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i: i + BATCH_SIZE]

        # Preprocess
        preprocessed = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # ✅ Extended token length (2048)
        inputs = tokenizer(
            preprocessed,
            truncation=True,
            max_length=1024,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                max_new_tokens=512,
                num_beams=3,           # a bit higher for better fluency
                early_stopping=True,
            )

        decoded = tokenizer.batch_decode(
            generated_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        postprocessed = ip.postprocess_batch(decoded, lang=tgt_lang)
        translations.extend(postprocessed)

        del inputs
        torch.cuda.empty_cache()

    return translations


# ✅ Initialize model and processor
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, quantization)
ip = IndicProcessor(inference=True)


In [ ]:
import pandas as pd

# Number of rows to preview
NUM_SAMPLES = 3  

def inspect_translations(df, src_lang, tgt_lang, model, tokenizer, ip, name):
    """
    Translates first few samples from a dataset's columns (instruction, input, output)
    and prints side-by-side English → Hindi translations.
    """
    print(f"\n\n=== {name} Dataset Translations ({src_lang} → {tgt_lang}) ===")
    
    # Select top samples
    df_sample = df.head(NUM_SAMPLES).copy()
    
    # Columns to translate
    columns_to_translate = ["instruction", "input", "output"]

    for col in columns_to_translate:
        print(f"\n🔹 Translating column: {col} ...")
        texts = df_sample[col].fillna("").astype(str).tolist()
        
        # Perform translation using your batch_translate() from earlier
        translated_texts = batch_translate(
            texts,
            src_lang=src_lang,
            tgt_lang=tgt_lang,
            model=model,
            tokenizer=tokenizer,
            ip=ip
        )
        df_sample[f"{col}_hi"] = translated_texts  # Store Hindi translations

    # --- Display results nicely ---
    for i, row in df_sample.iterrows():
        print("\n" + "=" * 100)
        print(f"📝 Row {i+1}")

        for col in columns_to_translate:
            print(f"\n{col.capitalize()}:")
            print(f"EN: {row[col]}")
            print(f"HI: {row[f'{col}_hi']}")
        print("=" * 100)

# Example usage:
inspect_translations(
    sentiment_df, "eng_Latn", "hin_Deva",
    en_indic_model, en_indic_tokenizer, ip, name="Sentiment"
)

inspect_translations(
    qa_df, "eng_Latn", "hin_Deva",
    en_indic_model, en_indic_tokenizer, ip, name="QA"
)


In [ ]:
from tqdm import tqdm
from transformers import AutoTokenizer

# Use a generic English tokenizer for counting (not IndicTrans)
count_tokenizer = AutoTokenizer.from_pretrained("t5-small")

max_len = 1024
token_counts = []

# Count tokens safely
for text in tqdm(qa_df["output"].fillna("").astype(str).tolist(), desc="Counting tokens"):
    tokens = count_tokenizer.encode(
        text,
        add_special_tokens=False,
        truncation=False
    )
    token_counts.append(len(tokens))

qa_df["output_token_count"] = token_counts

# --- Stats ---
total_rows = len(qa_df)
too_long = qa_df[qa_df["output_token_count"] > max_len]
max_tokens = qa_df["output_token_count"].max()

print(f"Total rows: {total_rows}")
print(f"Rows that would exceed {max_len} tokens: {len(too_long)}")
print(f"Percentage: {(len(too_long) / total_rows * 100):.2f}%")
print(f"Maximum token length in 'output': {max_tokens}")

# Optional: inspect a few long examples
display(too_long[["output", "output_token_count"]].head())


In [ ]:
# Define the max token limit
max_len = 1024

# Filter rows with acceptable token count
qa_df_filtered = qa_df[qa_df["output_token_count"] <= max_len].copy()

# Report how many rows were removed
removed_rows = len(qa_df) - len(qa_df_filtered)
print(f"✅ Removed {removed_rows} rows exceeding {max_len} tokens.")
print(f"Remaining rows: {len(qa_df_filtered)}")

# (Optional) Save the filtered dataframe for safe use
qa_df_filtered.to_csv("qa_df_filtered.csv", index=False)
print("💾 Saved filtered dataframe as 'qa_df_filtered.csv'")


In [ ]:
import pandas as pd
import json
import os
from tqdm import tqdm

# ================== CONFIG ==================
INPUT_CSV = "qa_df_filtered.csv"       # your source file
SAVE_CSV = "translated_dataset.csv"    # output file
PROGRESS_FILE = "translation_progress.json"

SRC_LANG = "eng_Latn"
TGT_LANG = "hin_Deva"

CHUNK_SIZE = 1000   # save every 1000 rows
BATCH_SIZE = 32     # already set above


# ================== RESUMABLE TRANSLATION (AUTO-SKIP) ==================
def translate_large_csv_safe(input_csv, output_csv, progress_file, src_lang, tgt_lang, model, tokenizer, ip):
    # --- Step 1: Load input dataset ---
    df = pd.read_csv(input_csv)
    print(f"Loaded {len(df)} rows from {input_csv}")

    # --- Step 2: Load existing translated data if present ---
    if os.path.exists(output_csv):
        translated_df = pd.read_csv(output_csv)
        done_rows = len(translated_df)
        print(f"Found existing translated CSV with {done_rows} rows ✅")
    else:
        translated_df = pd.DataFrame()
        done_rows = 0

    # --- Step 3: Resume from JSON progress if available ---
    start_idx = 0
    if os.path.exists(progress_file):
        with open(progress_file, "r") as f:
            progress = json.load(f)
            start_idx = progress.get("last_index", done_rows)
        print(f"Resuming from row {start_idx} 🔁")
    else:
        start_idx = done_rows

    # --- Step 4: Only process un-translated rows ---
    if start_idx >= len(df):
        print("🎉 All rows already translated — nothing to do!")
        return

    remaining_df = df.iloc[start_idx:].copy()

    # --- Step 5: Process and append chunks ---
    mode = "a" if os.path.exists(output_csv) else "w"
    header = not os.path.exists(output_csv)

    for start in range(0, len(remaining_df), CHUNK_SIZE):
        end = min(start + CHUNK_SIZE, len(remaining_df))
        chunk_df = remaining_df.iloc[start:end].copy()

        print(f"\n🔹 Translating rows {start_idx + start}–{start_idx + end} ...")

        for col in ["instruction", "input", "output"]:
            if col in chunk_df.columns:
                texts = chunk_df[col].fillna("").astype(str).tolist()
                chunk_df[f"{col}_hi"] = batch_translate(
                    texts, src_lang, tgt_lang, model, tokenizer, ip
                )

        # --- Step 6: Save incremental results ---
        chunk_df.to_csv(output_csv, mode=mode, header=header, index=False, encoding="utf-8-sig")
        mode, header = "a", False  # only first write includes header

        # --- Step 7: Update progress ---
        with open(progress_file, "w") as f:
            json.dump({"last_index": start_idx + end}, f)

        print(f"✅ Saved progress at row {start_idx + end}")

        # Free up memory
        del chunk_df
        import torch
        torch.cuda.empty_cache()

    print("\n🎉 Translation fully complete and saved safely to CSV!")


# ================== RUN TRANSLATION ==================
translate_large_csv_safe(
    INPUT_CSV, SAVE_CSV, PROGRESS_FILE,
    SRC_LANG, TGT_LANG, model, tokenizer, ip
)


In [ ]:
import pandas as pd
import json
import os
from tqdm import tqdm
import torch
model = en_indic_model
tokenizer = en_indic_tokenizer

# ================== CONFIG ==================
INPUT_PARQUET = "hf://datasets/FinGPT/fingpt-sentiment-cls/data/train-00000-of-00001-921f33f83a4110cb.parquet"
TEMP_CSV = "sentiment_df.csv"             # temporary CSV for streaming
SAVE_CSV = "sentiment_translated.csv"     # final translated dataset
PROGRESS_FILE = "sentiment_translation_progress.json"

SRC_LANG = "eng_Latn"
TGT_LANG = "hin_Deva"

CHUNK_SIZE = 1000
BATCH_SIZE = 32


# ================== STEP 1: Load parquet file ==================
print("📥 Loading sentiment dataset from HuggingFace Parquet...")
sentiment_df = pd.read_parquet(INPUT_PARQUET)
print(f"✅ Loaded {len(sentiment_df)} rows.")

# Save as CSV once for incremental translation process
sentiment_df.to_csv(TEMP_CSV, index=False)
print(f"💾 Saved a temporary working copy: {TEMP_CSV}")


# ================== STEP 2: Resumable translation function ==================
def translate_large_csv_safe(input_csv, output_csv, progress_file, src_lang, tgt_lang, model, tokenizer, ip):
    df = pd.read_csv(input_csv)
    print(f"Loaded {len(df)} rows from {input_csv}")

    # --- Step 2: Resume if previous results exist ---
    if os.path.exists(output_csv):
        translated_df = pd.read_csv(output_csv)
        done_rows = len(translated_df)
        print(f"Found existing partial output: {done_rows} rows ✅")
    else:
        translated_df = pd.DataFrame()
        done_rows = 0

    # --- Step 3: Resume from JSON progress ---
    start_idx = 0
    if os.path.exists(progress_file):
        with open(progress_file, "r") as f:
            progress = json.load(f)
            start_idx = progress.get("last_index", done_rows)
        print(f"Resuming from row {start_idx} 🔁")
    else:
        start_idx = done_rows

    # --- Step 4: Process un-translated rows only ---
    if start_idx >= len(df):
        print("🎉 All rows already translated — nothing to do!")
        return

    remaining_df = df.iloc[start_idx:].copy()
    mode = "a" if os.path.exists(output_csv) else "w"
    header = not os.path.exists(output_csv)

    # --- Step 5: Translate in chunks ---
    for start in range(0, len(remaining_df), CHUNK_SIZE):
        end = min(start + CHUNK_SIZE, len(remaining_df))
        chunk_df = remaining_df.iloc[start:end].copy()
        print(f"\n🔹 Translating rows {start_idx + start}–{start_idx + end} ...")

        for col in ["instruction", "input", "output"]:
            if col in chunk_df.columns:
                texts = chunk_df[col].fillna("").astype(str).tolist()
                chunk_df[f"{col}_hi"] = batch_translate(
                    texts, src_lang, tgt_lang, model, tokenizer, ip
                )

        # Save progress to CSV
        chunk_df.to_csv(output_csv, mode=mode, header=header, index=False, encoding="utf-8-sig")
        mode, header = "a", False

        # Save checkpoint
        with open(progress_file, "w") as f:
            json.dump({"last_index": start_idx + end}, f)

        print(f"✅ Saved progress at row {start_idx + end}")
        del chunk_df
        torch.cuda.empty_cache()

    print("\n🎉 Translation complete and safely saved to:", output_csv)


# ================== STEP 3: Run translation ==================
translate_large_csv_safe(
    TEMP_CSV, SAVE_CSV, PROGRESS_FILE,
    SRC_LANG, TGT_LANG, model, tokenizer, ip
)
